In [1]:
!ls -ltrh /dev/video*


crw-rw---- 1 root video 81, 0 May 12 07:45 /dev/video0


In [2]:
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
import ipywidgets
import numpy as np
import cv2
import os
print(cv2.__version__)

folder = "dnn"
proto = os.path.sep.join([folder, "deploy.prototxt"])
weight = os.path.sep.join([folder, "res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNet(proto, weight)
camera = USBCamera(width=640, height=360, capture_device=0)


image_widget = ipywidgets.Image(format='jpeg')
image_widget_blur = ipywidgets.Image(format='jpeg')

camera.running = True


4.1.1


In [3]:
def update_image(change):
    frame = change['new']
    image_widget.value = bgr8_to_jpeg(frame)
display(image_widget)

camera.observe(update_image, names='value')

Image(value=b'', format='jpeg')

In [4]:
def blur_face(image, factor=2):
    (h, w) = image.shape[:2]
    k_w = int(w / factor)
    k_h = int(h / factor)
    
    if k_w % 2 == 0:
        k_w -= 1
    if k_h % 2 == 0:
        k_h -= 1
        
    return cv2.GaussianBlur(image, (k_w, k_h), 0)

def update_image_blur(change):
    frame = change['new']
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300))
    net.setInput(blob)
    detections = net.forward()
    (h, w) = frame.shape[:2]
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            face = frame[startY:endY, startX:endX]
            face = blur_face(face)
            frame[startY:endY, startX:endX] = face
    image_widget_blur.value = bgr8_to_jpeg(frame)
display(image_widget_blur)
camera.observe(update_image_blur, names='value')

Image(value=b'', format='jpeg')